In [1]:
import torch
import math
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
import numpy as np
import pickle , gzip
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
device='cuda:0'

seed = 999
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [2]:
CLASS_CLOTHING = {0 :'T-shirt-top',
                  1 :'Trouser',
                  2 :'Pullover',
                  3 :'Dress',
                  4 :'Coat',
                  5 :'Sandal',
                  6 :'Shirt',
                  7 :'Sneaker',
                  8 :'Bag',
                  9 :'Ankle boot'}

In [3]:
def load_rotated_fmnist(left_out_idx=0):
    train_x = []
    test_x = []
    train_y = []
    test_y = []

    random = []
    for i in range(10):
        random.append(np.random.permutation(6000))
    
    for i in range(6):
        angle = 360- 15*i
        transform = transforms.Compose([transforms.RandomRotation(degrees = (angle,angle)),transforms.ToTensor()])
        cifar_train = datasets.FashionMNIST(root='data/',download=False, train=True, transform=transform)
        train_loader = torch.utils.data.DataLoader(dataset=cifar_train, batch_size=60000,shuffle=False)

        dataset = next(iter(train_loader))
        
        targets = dataset[1]
          
        data = dataset[0]
        
        
        
        data_x = []
        data_y = []
        
        #Picks 1000 images from each class
        for j in range(10):
           
            idx = targets==j
            jth_target = targets[idx]
            jth_data = data[idx]
            jth_data = jth_data[random[j]]
            
            
            sample_x = jth_data[:1000]
            sample_y = jth_target[:1000]
            
            data_x.append(sample_x)
            data_y.append(sample_y.float())
        
        data_x  = torch.cat(data_x).to(device)

        data_y  = torch.cat(data_y).to(device)

        
        if i!=left_out_idx:
            train_x.append(data_x)
            train_y.append(data_y)
        else:
            test_x = data_x
            test_y = data_y
    
    return train_x,train_y,test_x,test_y

###### MTAE 

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
     

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 500,bias=True).to(device),
            nn.ReLU(),
        ).to(device)

    def forward(self, x):
        out = self.encoder(x).to(device)
       
        return out

      
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
     
        self.decoder  = nn.Sequential(
                nn.Linear(500, 28*28,bias=True).to(device),
            
                nn.ReLU(),
            
            ).to(device)

    def forward(self, x):
        out = self.decoder(x).to(device)   
        return out



class MTAE:
    def __init__(self):
        super(MTAE,self).__init__()
          
        learning_rate = 0.007
        weight_decay = 3e-4

        self.Encoder = Encoder().to(device)

        self.Decoder = Decoder().to(device)

        self.params = list(self.Encoder.parameters()) + list(self.Decoder.parameters())

        self.optimizer = torch.optim.Adamax(params=self.params, lr=learning_rate,weight_decay=weight_decay)

        self.criterion = nn.MSELoss()
            
        
    def train(self,X,Y,domainId):
  
        self.optimizer.zero_grad()

        out = self.Encoder(X)
        
        out = self.Decoder(out)
                
        cost = self.criterion(out, Y)

        cost.backward()
  
        self.optimizer.step()
                
        self.optimizer.zero_grad()

        return cost.data.item()
    
    def get_features(self,X):
        model = self.Encoder
        model = model.eval()

        out = model(X)
        
        return out

In [4]:
class NeuralNet(nn.Module):

    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(512, 64).to(device)
        self.fc2 = nn.Linear(64, 10).to(device)

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x)).to(device)
        x = F.relu(self.fc2(x)).to(device)
        return x


    
def Neuron(feat_train, y_train,feat_test,y_test):
    model = NeuralNet().to(device)

    learning_rate = 0.002
    weight_decay = 0.001

    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate,weight_decay=weight_decay)

    criterion = nn.CrossEntropyLoss()
    batch_size = 40

    training_epochs = 70

    train_tensor = torch.utils.data.TensorDataset(feat_train, y_train) 
    train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

    total_batch = 50000 // batch_size

    for epoch in range(training_epochs):
        avg_cost = 0
        for i, (X, Y) in enumerate(train_loader):

            model.zero_grad()

            hypothesis = model(X)

            Y= Y.view(Y.shape[0])

            Y= Y.long()
            cost = criterion(hypothesis, Y)

            cost.backward()

            optimizer.step()
            model.zero_grad()
            avg_cost += (cost.data / total_batch)

        if epoch%10==0:
            print("Epoch: {}, averaged cost = {:.4f}".format(epoch + 1, avg_cost.item()))


    print('Learning Finished!')


    with torch.no_grad():
        correct = 0
        total = 0

        out = model(feat_test)
        _, predicted = torch.max(out, 1)
        
        y_test = y_test.long()
        correct += (predicted == y_test).sum().item()

        print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / len(y_test)))
        accuracy =correct/len(y_test)
    return accuracy


In [5]:

def linear_svm(feat_train,y_train,feat_test,y_test):#training_data,test_data):
  
    x = np.array(feat_train)
    y = np.array(y_train)

    linear_svm = OneVsRestClassifier(LinearSVC(C=3, max_iter=10000), n_jobs=-1)

    linear_svm.fit(x,y)


    train_outcome = linear_svm.predict(x)
    
    
    train_result = (train_outcome == y).sum()
    

    train_accuracy = train_result/len(x)
    
    print("Training Accuracy: %.8f" % train_accuracy)
    train_error = 1-train_accuracy
    print("Training Error: %.8f" % train_error)

    outcome = linear_svm.predict(np.array(feat_test))
    

    result = (outcome == y_test).sum()

    accuracy = result/len(outcome)
    print("Test accuracy: %.8f" % accuracy)
    test_error = 1-accuracy
    print("Test Error: %.8f" % test_error)

    return accuracy

In [1]:
def leave_one_out_training(dom):
    train_x,train_y,test_x,test_y = load_rotated_fmnist(left_out_idx=dom)
    batch_size=50
    
    mtae = MTAE()
    
    for epoch in range(10):

      
        x_train = []
        y_train = []

        random = np.random.permutation(10000)

        for i in range(5):
            x = train_x[i]
            x_permuted = x[random]

            y = train_y[i]
            y_permuted = y[random]
            
            x_train.append(x_permuted)
            y_train.append(y_permuted)
            
        
        x_train = torch.cat(x_train).to(device)

        x_train = x_train.view(5,10000,28*28)
        
        y_train  = torch.cat(y_train).to(device)
        
       
        for i in range(5):
            for j in range(5):
                avg_cost = 0
                for k in range(0,10000,50):
                    
                    left_x = x_train[i][k:k+50,:]
                    
                    right_x = x_train[j][k:k+50,:]
                    
                    avg_cost+= mtae.train(left_x,right_x,i)
                
                avg_cost = avg_cost/200
                
#             print(avg_cost)
    print("----------------------------------------")

    
    x_train = x_train.view(50000,28*28)
    
    train_feat = mtae.get_features(x_train)

    train_feat = train_feat.detach()

    feat_train = train_feat.view((50000,500))

    
    y_train = y_train.view(50000,1)
    
    x_test = test_x.view((10000, 28*28))

    feat = mtae.get_features(x_test)
    
    feat  = feat.detach()

    feat_test = feat.view((10000, 500))


    y_test = test_y.view(10000)

    y_test = y_test.long()

    print('-------------------------')


    linear_svm(feat_train.cpu(),y_train.cpu(),feat_test.cpu(),y_test.cpu())
#     accuracy= Neuron(feat_train.detach(),y_train.detach(),feat_test.detach(),y_test.detach())

    print("Accuracy :", accuracy/1)

In [ ]:
for dom in range(1,6):
    print("----------------------Domain.{}---------------------".format(dom))
    leave_one_out_training(dom)